In [1]:
import os, sys 
import pandas as pd
import numpy as np
os.chdir('../src')
import config
import glob
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import accuracy_score

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [67]:
sub_train = pd.read_csv(config.OUTPUT_FOLDER + 'stacker_xgb_7/sub_test.csv')
gt_train = pd.read_csv(config.DATA_FOLDER + 'gt_train.csv')
sub_train.rename(columns = {'target' : 'pred_target'}, inplace = True)
sub_train.rename(columns = {k: k + '_xgb' for k in config.mapping_dict.keys()}, inplace = True)

sub_train2 = pd.read_csv(config.OUTPUT_FOLDER + 'neural_clf/pred_test.csv')
sub_train2.rename(columns = {str(i) : {v:k for k,v in config.mapping_dict.iteritems()}[i] for i in np.arange(31)}, inplace = True)
sub_train2['target'] = sub_train2.drop('id', 1).idxmax(axis =1)
sub_train2.rename(columns = {'target' : 'pred_target2'}, inplace = True)
sub_train2.rename(columns = {k: k + '_nn_clf' for k in config.mapping_dict.keys()}, inplace = True)

In [68]:
df = sub_train.merge(sub_train2, on = 'id')
assert len(df) == config.LEN_DFS['test']

In [69]:
for c in config.mapping_dict.keys():
    df[c] = np.sqrt(df[c + '_xgb'] * df[c + '_nn_clf'])
df['geomean_target'] = df[config.mapping_dict.keys()].idxmax(axis = 1)

In [71]:
df['fname'] = df['id'] + '.wav'
df['label'] = df['geomean_target'] 
df[['fname', 'label']].to_csv('geomean_clf_nn_and_stacker_7.csv', index = False)

In [62]:
accuracy_score(df['target'].values, df['pred_target'].values)

0.96659860194058989

In [63]:
accuracy_score(df['target'].values, df['pred_target2'].values)

0.96661350662513223

In [64]:
accuracy_score(df['target'].values, df['geomean_target'].values)

0.96686688626235229

In [65]:
accuracy_score(df['target'].values, df['mean_target'].values)

0.96679236283964054

In [66]:
df['geomean_target'].value_counts()

silence    2622
off        2445
on         2430
stop       2425
three      2402
no         2376
up         2367
two        2364
go         2362
six        2357
seven      2351
one        2346
yes        2345
zero       2343
right      2340
four       2337
nine       2333
eight      2332
left       2331
five       2329
down       2325
dog        1746
house      1745
wow        1736
happy      1735
cat        1733
marvin     1732
bed        1730
sheila     1719
bird       1700
tree       1655
Name: geomean_target, dtype: int64